# Reproducing results from the Multi-News paper

In this notebook we will try to reproduce the results from  the [Multi-News](https://www.aclweb.org/anthology/P19-1102/) [(GitHub)](https://github.com/Alex-Fabbri/Multi-News). 


They use the [OpenNMT](https://opennmt.net/) neural machine translation system, but adapted it some places.

The main steps of a OpenNMT-pipeline are
- Preprocessing 
- Model training
- Translate

You can find the documentation [here](https://opennmt.net/OpenNMT-py/quickstart.html#step-1-preprocess-the-data).


See also the python implementation of OpenNMT [here](https://github.com/OpenNMT/OpenNMT-py).

## Preparing Data

The data consists of parallel source (src) and target (tgt) data containing one sentence per line with tokens separated by a space.
We won't need to do preprocessing on our own, since preprocessed data is already provided

In [1]:
!dir

code  data  output  README.md  Train_BERT_HiPMAP.ipynb	Train_HiPMAP.ipynb


In [1]:
!pip install torchtext nltk opencv-python transformers==3.1.0

In [3]:
!python code/HiPMAP/preprocess.py \
    -train_src ../news-opinion-summarization/data/multi_news/raw/train.src.cleaned \
    -train_tgt ../news-opinion-summarization/data/multi_news/raw/train.tgt \
    -valid_src ../news-opinion-summarization/data/multi_news/raw/val.src.cleaned \
    -valid_tgt ../news-opinion-summarization/data/multi_news/raw/val.tgt \
    -save_data ../news-opinion-summarization/data/multi_news/newser_sent_500/newser_sents \
    -src_seq_length 10000 \
    -tgt_seq_length 10000 \
    -src_seq_length_trunc 500 \
    -tgt_seq_length_trunc 300 \
    -dynamic_dict \
    -share_vocab \
    -max_shard_size 10000000


Please backup existing pt file: ../news-opinion-summarization/data/multi_news/newser_sent_500/newser_sents.train*.pt, to avoid tampering!


## Train the Model



In [9]:
!export THC_CACHING_ALLOCATOR=0


In [3]:
%%time
!CUDA_VISIBLE_DEVICES=2,3,0,1 python code/HiPMAP/train.py \
    -save_model output/summarisation/model_newser_polarity/Okt17_ \
    -data ../news-opinion-summarization/data/multi_news/final_preprocessed/final \
    -copy_attn -accum_count 5\
    -global_attention mlp \
    -word_vec_size 128 \
    -rnn_size 512  -layers 1 \
    -encoder_type brnn \
    -train_steps 20000 \
    -max_grad_norm 4 \
    -dropout 0. \
    -batch_size 2 \
    -optim adagrad \
    -learning_rate 0.15 \
    -adagrad_accumulator_init 0.1 \
    -reuse_copy_attn \
    -copy_loss_by_seqlength \
    -bridge \
    -seed 777 \
    -world_size 1  \
    -gpu_ranks 0 1 2 3 \
    -save_checkpoint_steps 100 \
    -train_from output/summarisation/model_newser_polarity/Okt17__step_19900.pt
    #-model_type fp16 \


2020-10-23 07:37:17.940309: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-10-23 07:37:17.940366: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-10-23 07:37:17.940377: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
[2020-10-23 07:37:23,943 INFO] Loading checkpoint from output/summarisation/model_newser_polarity/Okt17__step_19900.pt
[2020-10-23 07:37:26,957 INFO] Loa

In [4]:
!nvidia-smi

Mon Oct 19 18:05:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 31%   44C    P8    16W / 250W |      0MiB / 11016MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:21:00.0 Off |                  N/A |
| 30%   42C    P8    23W / 250W |      0MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [8]:
print('test')

test
